# Sparkify Data Science Nanodegree Capstone Project

* The goal of this data science project is to predict user churn, i.e., identify what causes users to create a subscription or cancel it respectively.
* Instead of using the pandas library, we instead use the Apache Spark framework, which is build for big data analysis running on clusters.
* The dataset used is only a fraction of the original dataset, which is about 12GB large and stored on AWS S3 (in a first step we use the small dataset; later we'll run this notebook on an AWS EMR Cluster)

# Set up Spark and other Dependencies

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, split, trim, expr, datediff, from_unixtime

from pyspark.ml.feature import RegexTokenizer, CountVectorizer, \
    IDF, StringIndexer

import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [4]:
spark = SparkSession \
    .builder \
    .appName("sparkify") \
    .getOrCreate()

In [5]:
spark

In [183]:
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

Run the cell below to increase width of notebook. Will be needed for SQL data operations.

In [184]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
# display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Data Exploration

To learn more about the users the follwoing questions need to answered:


## Who are the Users?

* What gender do users have?
* Where do they live?
* What are their favorite artists/songs?

## How do they interact with the Platform?

### Platform Usage

* How often do they use the platform?
* Do users who cancel their subscription don't use the platform or do they and might not like it?
* How long is an average session per user?
* What devices do they use to access the platform?

### Subscription Behavior

* How often do users switch their subscription level (paid/free)
* Is there a leading indicator that a user might cancel his paid subscriptions, e.g., high count of thumbs down



In [185]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [186]:
user_log.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

The timestamp format is not in a readable format. By creating the user defined function `get_datetime` and applying this function onto the ts column we obtain the dateTime column which has a convenient format

In [187]:
# get_datetime = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))

# user_log = user_log.withColumn("ts_iso", get_datetime(user_log.ts)) \
#             .withColumn("registration_iso", get_datetime(user_log.registration))

# use pysparks built-in function to handel unix timestamps (divide by 1000 to convert from milliseconds)
user_log = user_log.withColumn("ts_iso", from_unixtime(user_log.ts/1000)) \
                .withColumn("registration_iso", from_unixtime(user_log.registration/1000))

The location column contains both city and state, we want to isolate both of them into seperate columns

In [188]:
user_log = user_log.withColumn('city', trim(split(user_log['location'], ',').getItem(0))) \
            .withColumn('state', trim(split(user_log['location'], ',').getItem(1)))

The userAgent columns contains information about which end-device was used to access the platform. Therfore it is usefull to learn something about the platform's users

In [189]:
user_log = user_log.withColumn('userDevice', expr("CASE WHEN lower(userAgent) LIKE '%windows%' THEN 'pc'"+
                                       "WHEN lower(userAgent) LIKE '%macintosh%' THEN 'mac'" +
                                        "WHEN lower(userAgent) LIKE '%linux%' THEN 'pc'" +
                                        "WHEN lower(userAgent) LIKE '%iphone%' THEN 'mobile'" +
                                        "WHEN lower(userAgent) LIKE '%ipad%' THEN 'mobile'" +
                                        "ELSE 'other'" +
                                        "END"))

In [190]:
user_log.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', ts_iso='2018-10-01 02:01:57', registration_iso='2018-09-29 00:22:42', city='Bakersfield', state='CA', userDevice='pc')

Since we are interested in predicting user behavior, actions recorded in the dataset need to be attributed to a userid. For records without a userId we are not able to draw conclusions on the behaviour.

In [191]:
print('{} records in the dataset have no userId. They will be dropped'.format(user_log.filter(user_log.userId == '').count()))
user_log = user_log.where(user_log.userId != '')
print('{} records in the dataset have no userId.'.format(user_log.filter(user_log.userId == '').count()))


8346 records in the dataset have no userId. They will be dropped
0 records in the dataset have no userId.


## Leveraging PySpark's SQL Function for Data Exploration

In [192]:
# create temporary view against which SQL queries can be run
user_log.createOrReplaceTempView("user_log_table")

### Create a table that contains user subscription behavior

Idea of the query below:

1. `errors_tab`
    * Assign each entry row number to maintain order of events (have consecutive entries with exact same timestamp)
    * Some actions are not realized because an error follows in the logs. Thus if entry $t+1$ has page "Error", set a flag for entry $t$ (Note that we want to keep the actual errors themselves, since they might be an indicator for users terminating their subscription, if they encounters a large number of errors during their paid regime)

2. `records_tab`
    * When level at $t$ is different from $t-1$ a new regime starts. Set a flag
    * When level at $t$ is "paid" and level at $t+1$ is "free", the regime at $t$ is cancelled. Set a flag
    * Choose only actions that don't result in errors

3. `regime_tab`
    * Sum up the flag set in the `records_tab`. Whenever the level changes, the `regime_count` increases by 1

4. `regime_dates`
    * The `regime_start` in the first regime is per assumption the registration date, as we don't have other data available. In later regimes $n>1$ we'll take the timestamp of the first record of regime $n$
    * In the last regime, $N$, the `regime_end` is the max. timestamp in the dataset. Else it's the timestamp of the latest record in the particular regime $n<N$

5. Final Result
    * Collect the pre-processed data
    * Obtain aggregate values per regime

**Note:** might want to break the pretabs down into seperate SQL query commands and tables to reduce complexity of statement

In [193]:
spark.sql("""
            SELECT * FROM user_log_table
            WHERE userId = 100001
            ORDER BY userId, ts
        """).show(5000)

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------------------+-------------+---------+--------------------+------+-------------+--------------------+------+-------------------+-------------------+--------------------+-----+----------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|                page| registration|sessionId|                song|status|           ts|           userAgent|userId|             ts_iso|   registration_iso|                city|state|userDevice|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------------------+-------------+---------+--------------------+------+-------------+--------------------+------+-------------------+-------------------+--------------------+-----+----------+
|                null|Logged In|  Delaney|     F|            0|   Pere

In [194]:
user_regime = spark.sql("""
                        with errors_tab AS(
                        SELECT
                            userId, auth, page, level, ts/1000 ts, registration/1000 registration, userDevice,
                            row_number() OVER(PARTITION BY userId ORDER BY userId, ts) record_number,
                            CASE WHEN LEAD(page) OVER(PARTITION BY userId ORDER BY userId, ts) = 'Error' THEN 1 ELSE 0 END leads_to_error
                        FROM user_log_table
                        ),


                        records_tab as(
                        SELECT
                            userId, page, level, ts, registration, userDevice, record_number,
                            CASE WHEN auth = 'Cancelled' THEN ts END date_fully_cancelled,
                            CASE WHEN LAG(level) OVER(PARTITION BY userId ORDER BY record_number) != level THEN 1 ELSE 0 END regime_change,
                            CASE WHEN LEAD(level) OVER(PARTITION BY userId ORDER BY record_number) = 'free' and level = 'paid' THEN 1 ELSE 0 END downgrade_regime
                        FROM errors_tab
                        WHERE leads_to_error = 0
                        ),


                        regime_tab AS(
                        SELECT userId, page, level, ts, registration, userDevice, record_number, date_fully_cancelled, downgrade_regime,
                            SUM(regime_change) OVER(PARTITION BY userId ORDER BY record_number)+1 regime_count
                        FROM records_tab),


                        regime_dates AS(            
                        SELECT userId, page, level, userDevice, record_number, FROM_UNIXTIME(date_fully_cancelled) date_fully_cancelled, downgrade_regime, regime_count,
                            CASE
                                WHEN regime_count = 1 THEN FROM_UNIXTIME(registration)
                                WHEN regime_count > 1 THEN FROM_UNIXTIME(first_value(ts) OVER(PARTITION BY userId, regime_count))
                            END regime_start,
                            
                            CASE
                                WHEN regime_count = last_value(regime_count) OVER(PARTITION BY userId ORDER BY ts) AND date_fully_cancelled IS NULL THEN FROM_UNIXTIME((select max(ts) FROM records_tab))
                                WHEN regime_count = last_value(regime_count) OVER(PARTITION BY userId ORDER BY ts) AND date_fully_cancelled IS NOT NULL THEN FROM_UNIXTIME(date_fully_cancelled)
                                ELSE FROM_UNIXTIME(last_value(ts) OVER(PARTITION BY userId, regime_count))
                            END regime_end
                        FROM regime_tab)

                        SELECT
                            DISTINCT userId,
                            level,
                            regime_count,
                            DATE(regime_start) regime_start,
                            DATE(regime_end) regime_end,
                            DATEDIFF(day, regime_start, regime_end) regime_lenght,
                            SUM(downgrade_regime) OVER(PARTITION BY userId, regime_count) = 1 regime_downgraded,
                            DATE(date_fully_cancelled) date_fully_cancelled,
                            count_if(page = 'NextSong') OVER(PARTITION BY userId, regime_count) next_song,
                            count_if(page = 'Thumbs Up') OVER(PARTITION BY userId, regime_count) thumbs_up,
                            count_if(page = 'Thumbs Down') OVER(PARTITION BY userId, regime_count) thumbs_down,
                            count_if(page = 'Add Friend') OVER(PARTITION BY userId, regime_count) add_friend,
                            count_if(page = 'Add to Playlist') OVER(PARTITION BY userId, regime_count) add_to_playlist,
                            count_if(page = 'Save Settings') OVER(PARTITION BY userId, regime_count) save_settings,
                            count_if(page = 'Error') OVER(PARTITION BY userId, regime_count) error,
                            count_if(page = 'Help') OVER(PARTITION BY userId, regime_count) help,
                            count_if(page = 'Roll Advert') OVER(PARTITION BY userId, regime_count) roll_advert,
                            userDevice
                        FROM regime_dates
                        ORDER BY userId, regime_count
                        
                        """)
# store results in temporary view: user_regime_table
user_regime.createOrReplaceTempView("user_regime_table")
user_regime.show(100)

+------+-----+------------+------------+----------+-------------+-----------------+--------------------+---------+---------+-----------+----------+---------------+-------------+-----+----+-----------+----------+
|userId|level|regime_count|regime_start|regime_end|regime_lenght|regime_downgraded|date_fully_cancelled|next_song|thumbs_up|thumbs_down|add_friend|add_to_playlist|save_settings|error|help|roll_advert|userDevice|
+------+-----+------------+------------+----------+-------------+-----------------+--------------------+---------+---------+-----------+----------+---------------+-------------+-----+----+-----------+----------+
|    10| paid|           1|  2018-09-28|2018-12-03|           65|            false|                null|      673|       37|          4|        12|              9|            1|    0|   1|          1|       mac|
|   100| paid|           1|  2018-09-26|2018-12-03|           67|             true|                null|     1604|       84|         14|        32|     

### Aggregate the `user_regime_table` data per User

The aggregate table should have one row per user which contains all necessary information and can be used for ML model input.

In [196]:
spark.sql("""
            SELECT
                userId,
                count_if(level = 'paid') paid_regimes,
                count_if(level = 'free') free_regimes,
                sum(regime_lenght) membership_days,
                count_if(regime_downgraded = 'true') downgraded_regimes,
                max(date_fully_cancelled) IS NOT NULL fully_cancelled,
                sum(next_song) next_song,
                sum(thumbs_up) thumbs_up,
                sum(thumbs_down) thumbs_down,
                sum(add_friend) add_friend,
                sum(add_to_playlist) add_to_playlist,
                sum(save_settings) save_settings,
                sum(error) error,
                sum(help) help,
                sum(roll_advert) roll_advert,
                count_if(userDevice = 'mac')>0 device_mac,
                count_if(userDevice = 'pc')>0 device_pc,
                count_if(userDevice = 'mobile')>0 device_mobile
            FROM
                user_regime_table
            GROUP BY
                userId
            ORDER BY
                userId
            """).show()

+------+------------+------------+---------------+------------------+---------------+---------+---------+-----------+----------+---------------+-------------+-----+----+-----------+----------+---------+-------------+
|userId|paid_regimes|free_regimes|membership_days|downgraded_regimes|fully_cancelled|next_song|thumbs_up|thumbs_down|add_friend|add_to_playlist|save_settings|error|help|roll_advert|device_mac|device_pc|device_mobile|
+------+------------+------------+---------------+------------------+---------------+---------+---------+-----------+----------+---------------+-------------+-----+----+-----------+----------+---------+-------------+
|    10|           1|           0|             65|                 0|          false|      673|       37|          4|        12|              9|            1|    0|   1|          1|      true|    false|        false|
|   100|           2|           2|            122|                 1|          false|     2910|      159|         28|        55|    

In [50]:
user_log.select("auth").groupby("auth").count().show()

+---------+------+
|     auth| count|
+---------+------+
|Cancelled|    52|
|Logged In|278102|
+---------+------+



In [51]:
user_log.select("gender").groupby("gender").count().show()

+------+------+
|gender| count|
+------+------+
|     F|154578|
|     M|123576|
+------+------+



In [52]:
user_log.select("level").groupby("level").count().orderBy("count", ascending=False).show()


+-----+------+
|level| count|
+-----+------+
| paid|222433|
| free| 55721|
+-----+------+



In [53]:
user_log.select("page").groupby("page").count().orderBy("count", ascending=False).show()

+--------------------+------+
|                page| count|
+--------------------+------+
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Home| 10082|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|         Roll Advert|  3933|
|              Logout|  3226|
|         Thumbs Down|  2546|
|           Downgrade|  2055|
|            Settings|  1514|
|                Help|  1454|
|             Upgrade|   499|
|               About|   495|
|       Save Settings|   310|
|               Error|   252|
|      Submit Upgrade|   159|
|    Submit Downgrade|    63|
|              Cancel|    52|
|Cancellation Conf...|    52|
+--------------------+------+



In [ ]:
user_log.select("userDevice").groupby("userDevice").count().orderBy("count", ascending=False).show()

In [ ]:
spark.sql('''
        SELECT artist, count(*) as count
        FROM user_log_table
        WHERE artist is not null
        GROUP BY artist
        ORDER BY count(*) DESC
        ''').show()

In [ ]:
spark.sql('''
        SELECT song, count(*) as count
        FROM user_log_table
        WHERE song is not null
        GROUP BY song
        ORDER BY count(*) DESC
        ''').show()

In [ ]:
spark.sql('''
        SELECT artist, song, count(*) as count
        FROM user_log_table
        WHERE
            song is not null
            AND artist is not null
        GROUP BY artist, song
        ORDER BY artist, count(*) DESC
        ''').show()

In [ ]:
df_membership_days = spark.sql('''
                                SELECT userId, max(membership_days)
                                FROM user_log_table
                                GROUP BY userId
                                ORDER BY max(membership_days) DESC
                                ''').toPandas()

In [ ]:
df_membership_days.plot(kind='hist', title='Membership in Days', legend=False, figsize=(10,6));

In [ ]:
spark.sql('''
            SELECT userId, max(membership_days)
            FROM user_log_table
            WHERE page = 'Cancellation Confirmation'
            GROUP BY userId
            ORDER BY max(membership_days) DESC
            ''').show()

In [ ]:
spark.sql('''
            with pretab as(
            SELECT userId, max(membership_days)
            FROM user_log_table
            WHERE page = 'Cancellation Confirmation'
            GROUP BY userId
            ORDER BY max(membership_days) DESC
            )
            SELECT pretab.userId, count(*) records, max(membership_days)
            FROM
                user_log_table
                INNER JOIN pretab on pretab.userId = user_log_table.userId
            GROUP BY pretab.userId
            ORDER BY max(membership_days) desc
            ''').show()

In [ ]:
df_cancelled_userId = user_log.select('userId').filter(user_log.page == 'Cancellation Confirmation').drop_duplicates()

In [ ]:
df_cancelled_userId.createOrReplaceTempView("cancelled_users_table")

In [ ]:
spark.sql('''
            with is_home as(
            SELECT
                userID, page, ts,
                CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home                
            FROM user_log_table
            WHERE
                (page = 'NextSong') or (page = 'Home')
            ),
            cum_sum as (
            SELECT *, sum(is_home) OVER(PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as period
            FROM is_home
            )
            SELECT AVG(count_results)
            FROM (
                SELECT COUNT(*) AS count_results FROM cum_sum
                GROUP BY userID, period, page HAVING page = 'NextSong'
            ) as counts
                
            ''').show()

In [ ]:
spark.sql('''
            with is_home as(
            SELECT
                userId, auth, page, ts,
                CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home                
            FROM user_log_table
            WHERE
                (page = 'NextSong') or (page = 'Home')
            ),
            cum_sum as (
            SELECT *, sum(is_home) OVER(PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as period
            FROM is_home
            )
            SELECT "cancelled_users" as user_state, AVG(count_results) as 
            FROM (
                SELECT COUNT(*) AS count_results FROM cum_sum WHERE userId in (select userId from cancelled_users_table)
                GROUP BY userID, period, page HAVING page = 'NextSong'
            ) as counts
            
            UNION
            
            SELECT "active_users" as user_state, AVG(count_results)
            FROM (
                SELECT COUNT(*) AS count_results FROM cum_sum WHERE userId not in (select userId from cancelled_users_table)
                GROUP BY userID, period, page HAVING page = 'NextSong'
            ) as counts
            
            
            ''').show()

In [ ]:
spark.sql('''
            with is_home as(
            SELECT
                userId, auth, page, ts,
                CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home                
            FROM user_log_table
            WHERE
                (page = 'NextSong') or (page = 'Home')
            ),
            cum_sum as (
            SELECT *, sum(is_home) OVER(PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as period
            FROM is_home
            )
            
            select userId,  from cum_sum
            
                
            ''').show()

In [ ]:
spark.sql('''
            SELECT userId, auth, ts, ts_iso, page, sessionId
            FROM user_log_table
            WHERE (page = 'Home' or page = 'NextSong')
                and userId=10
            ORDER BY userId, ts asc
            ''').show(1000)

### Todo

To understand how users interact with the platform, need to know:

* Songs per session
* Session count
* Session frequency
    * Sessions over membership days
    * Avg time between sessions

Need to differentiate between cancelled and active users (makes sense to add `cancelled` indicator to user_log dataframe)

# Feature Engineering

* User inactivity: if a user only listens to a handful of songs per month or no songs at all, a cancellation might be likely
    * The ratio of songs listend over time being a paying client might be a good predictor
* Number of thumbs up/down they gave before cancelling

# Modeling

In version 3.3.2 spark supports the following classification algorithms:

* Logistic regression
    * Binomial logistic regression
    * Multinomial logistic regression
* Decision tree classifier
* Random forest classifier
* Gradient-boosted tree classifier
* Multilayer perceptron classifier
* Linear Support Vector Machine
* One-vs-Rest classifier (a.k.a. One-vs-All)
* Naive Bayes
* Factorization machines classifier

We'll choose the four most popular: Logistic regression, Random forest classifier, Gradient-boosted tree classifier and Linear Support Vector Machine